In [3]:
import pandas as pd

In [6]:
df = pd.DataFrame([[1, 2, 3, 4, 5], [3, 4, 1, 2, 5]], columns=list('ABCDE'))
df

,A,B,C,D,E
0,1,2,3,4,5
1,3,4,1,2,5


In [13]:
df2 = pd.DataFrame([[6, 7]], columns=['B', 'E'])
df.append(df2)
df

,A,B,C,D,E
0,1,2,3,4,5
1,3,4,1,2,5


In [11]:
df2 = pd.Series([7], index=['C'])
df.append(df2, ignore_index=True)

,A,B,C,D,E
0,1.0,2.0,3.0,4.0,5.0
1,3.0,4.0,1.0,2.0,5.0
2,NaN,NaN,7.0,NaN,NaN


In [38]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import RegexpTokenizer

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

def recommend_books(title, genre):
    df = pd.read_csv('DATA/library/bnb/records.csv')

    # filtrate titles
    df2 = df[df['Title'].notnull()]

    # add specified title
    df3 = pd.DataFrame([[title, genre]], columns=['Title', 'Genre'])
    df2 = df2.append(df3)

#     df2 = df2[df2['Genre'].str.contains(genre)]

    # df with genre filtrating and without
    
    ### def get_recommend_title()
    
    # create cleaned titles to calculate cosine similarity
    df2['format_title'] = df2['Title'].str.lower()
    df2['format_title'] = df2['format_title'].apply(func=remove_punctuation)
    df2.drop_duplicates(subset='format_title', keep='first', inplace=True)
    
    data = df2
    
    # create new indices for interacting in all data frames
    data.reset_index(level=0, inplace=True)

    idx = data.index[data['Title'] == title][0]

    tf = TfidfVectorizer(analyzer='word', ngram_range=(2, 2),
                         min_df=1, stop_words='english')
    tfidf_matrix = tf.fit_transform(data['format_title'])

    # get matrix for specified title
    tt_matrix = tfidf_matrix[idx]

    sim_arr = cosine_similarity(tt_matrix, tfidf_matrix)[0]

    # enumerate array elements to correspond books indices in data
    idx_sim = list(enumerate(sim_arr))

    idx_sim = sorted(idx_sim, key=lambda x: x[1], reverse=True)
    idx_sim = idx_sim[1:6]
    print(idx_sim)

    book_indices = [i[0] for i in idx_sim]
    
    ### return List[(title, similarity)]
    
    recommends = data['Title'].iloc[book_indices]

    return recommends


def test_funcs():
    title, genre = 'Don Juan Demarico', ''
    recs = recommend_books(title, genre)
    print(list(recs))

test_funcs()

[(1346, 0.41919640810025893), (533, 0.24222625712430557), (537, 0.24222625712430557), (1480, 0.24222625712430557), (518, 0.24023538316866705)]
['Don Juan', "Byron's Don Juan", "Byron's Don Juan, etc", 'Byron, Don Juan', 'Don Juan. Cantos I to V']
